**Run queries on triplestore**

The data from the last section has been loaded into a triplestore at http://138.197.180.196:3030/

Queries are made using a language called SPARQL and can be either submitted manually within the web interface, or requested programmatically as below

Here are some basic Python functions to send the request across to the triplestore and retrieve a data table

In [16]:
import pandas
import pydash
import requests

def value_extract(row, col):

    """Extract dictionary values."""

    return pydash.get(row[col], "value")

def sparql_query(query):

    """Send sparql request, and formulate results into a dataframe."""

    r = requests.post('http://138.197.180.196:3030/test-data/sparql', data={'query': query}, verify=False)  
    data = pydash.get(r.json(), "results.bindings")
    data = pandas.DataFrame.from_dict(data)
    for x in data.columns:
        data[x] = data.apply(value_extract, col=x, axis=1)
    return data

The query to return everything: select * where { ?a ?b ?c }

Importantly even though we are using Wikidata identifiers, there is no interaction with Wikidata itself

The triplestore is only aware of the eight statements which we provided earlier

In [17]:
dataframe = sparql_query('select * where { ?a ?b ?c } ')
dataframe.head(10)

,a,b,c
0,http://www.wikidata.org/entity/Q2316927,http://www.wikidata.org/entity/P31,http://www.wikidata.org/entity/Q11424
1,http://www.wikidata.org/entity/Q2316927,http://www.wikidata.org/entity/P57,http://www.wikidata.org/entity/Q446294
2,http://my-archive.org/acetate_print_of_simple_men,http://www.wikidata.org/entity/P31,http://my-archive.org/film_item
3,http://my-archive.org/acetate_print_of_simple_men,http://www.wikidata.org/entity/P186,http://www.wikidata.org/entity/Q124686
4,http://www.wikidata.org/entity/Q455552,http://www.wikidata.org/entity/P31,http://www.wikidata.org/entity/Q11424
5,http://www.wikidata.org/entity/Q455552,http://www.wikidata.org/entity/P57,http://www.wikidata.org/entity/Q446294
6,http://my-archive.org/polyester_print_of_amateur,http://www.wikidata.org/entity/P31,http://my-archive.org/film_item
7,http://my-archive.org/polyester_print_of_amateur,http://www.wikidata.org/entity/P186,http://www.wikidata.org/entity/Q188245


At its most basic level, SPARQL works by pattern matching

Say we want to return everything which "is a" "film", we would substitute "b" and "c" for these values

Remember the triangle brackets around any URL in the query!

In [18]:
dataframe = sparql_query('select * where { ?a <http://www.wikidata.org/entity/P31> <http://www.wikidata.org/entity/Q11424> } ')
dataframe.head(10)

,a
0,http://www.wikidata.org/entity/Q2316927
1,http://www.wikidata.org/entity/Q455552


Calling the result "a" is likely to cause confusion, so we can rename the result column to "film_work"

In [19]:
dataframe = sparql_query('select ?film_work where { ?film_work <http://www.wikidata.org/entity/P31> <http://www.wikidata.org/entity/Q11424> } ')
dataframe.head(10)

,film_work
0,http://www.wikidata.org/entity/Q2316927
1,http://www.wikidata.org/entity/Q455552


We can also ask questions about specific items, for example who directed http://www.wikidata.org/entity/Q455552?

In [20]:
dataframe = sparql_query('select ?director where { <http://www.wikidata.org/entity/Q455552> <http://www.wikidata.org/entity/P57> ?director } ')
dataframe.head(10)

,director
0,http://www.wikidata.org/entity/Q446294


At this stage, we may realise that our ability to make interesting queries is limited because the initial data statements were minimal

For instance, we have no formal link between the "film work" and "film item" (eg http://my-archive.org/acetate_print_of_simple_men and http://www.wikidata.org/entity/Q2316927)

To fix can create a new property http://my-archive.org/has_film_item and link our "film works" and "film items" together

```
<http://www.wikidata.org/entity/Q2316927> <http://my-archive.org/has_film_item> <http://my-archive.org/acetate_print_of_simple_men> .
<http://www.wikidata.org/entity/Q455552> <http://my-archive.org/has_film_item> <http://my-archive.org/polyester_print_of_amateur> .
```

We are currently getting responses to our questions as URLs, so then need to consult the URL itself to find out "what it is"

We can fix this situation by adding labels

Again this has no connection with what Wikidata calls these things, we can name them whatever we like in our own graph

```
http://www.wikidata.org/entity/Q446294 has label "Hal Hartley"
http://www.wikidata.org/entity/Q2316927 has label "Simple Men"
http://www.wikidata.org/entity/Q455552 has label "Amateur"
http://my-archive.org/acetate_print_of_simple_men has label "Acetate Print of Simple Men"
http://my-archive.org/polyester_print_of_amateur has label "Polyester Print of Amateur"	
http://www.wikidata.org/entity/Q11424 has label "Film Work"
http://my-archive.org/film_item has label "Film Item"
http://www.wikidata.org/entity/Q124686 has label "Acetate"
http://www.wikidata.org/entity/Q188245 has label "Polyester"
```

Luckily for us, "has label" is already an established property (http://www.w3.org/2000/01/rdf-schema#label) which we can reuse

```
<http://www.wikidata.org/entity/Q446294> <http://www.w3.org/2000/01/rdf-schema#label> "Hal Hartley" .
<http://www.wikidata.org/entity/Q2316927> <http://www.w3.org/2000/01/rdf-schema#label> "Simple Men" .
<http://www.wikidata.org/entity/Q455552> <http://www.w3.org/2000/01/rdf-schema#label> "Amateur" .
<http://my-archive.org/acetate_print_of_simple_men> <http://www.w3.org/2000/01/rdf-schema#label> "Acetate Print of Simple Men" .
<http://my-archive.org/polyester_print_of_amateur> <http://www.w3.org/2000/01/rdf-schema#label> "Polyester Print of Amateur" .
<http://www.wikidata.org/entity/Q11424> <http://www.w3.org/2000/01/rdf-schema#label> "Film Work" .
<http://my-archive.org/film_item> <http://www.w3.org/2000/01/rdf-schema#label> "Film Item" .
<http://www.wikidata.org/entity/Q124686> <http://www.w3.org/2000/01/rdf-schema#label> "Acetate" .
<http://www.wikidata.org/entity/Q188245> <http://www.w3.org/2000/01/rdf-schema#label> "Polyester" .
```

Notice that our labels are not URLs, they are just the words wrapped in quotation marks

These are what is known as "literals"